Student Name:Tom Keane

Student ID:20214537

In [2]:
import csv

In [3]:
#   Matrix class has two variables;
#        value is a nested tuple containing the matrix' values
#        dimens is a string like so: "mXn" where m & n are the matrix  dimension values
#         size gives the dimensions as a tuple

# Generally, most basic operations use A & B as the operands and C as the result,
#  ie. A + B = C

In [4]:
class Matrix:
    def __init__(self, value=None):
        if not value == None:
            self.value = value
            self.size = ( (len(value)), (len(value[0])) )
            self.dimens = str(self.size[0])+'X' + str(self.size[1])
    

    def __add__(self,other):
        if self.dimens != other.dimens:
            raise Exception("add error")
            return
        A = self.value
        B = other.value
        C = []
        for i in range(len(A)):
            C_i = []
            for j in range(len(A[i])):
                C_i.append(A[i][j] + B[i][j])
            C.append(tuple(C_i))
        return Matrix(tuple(C))
    
    def __sub__(self,other):
        if self.dimens != other.dimens:
            raise Exception("sub error")
            return
        A = self.value
        B = other.value
        C = []
        for i in range(len(A)):
            C_i = []
            for j in range(len(A[i])):
                C_i.append(A[i][j] - B[i][j])
            C.append(tuple(C_i))
        return Matrix(tuple(C))

    def __mul__(self,other):
        A = self.value
        C = []
        if not isinstance(other, Matrix):
            return self.scalarMultiplication(other)
        
        if not self.canMultiply(other):
            raise Exception("mul error")
            return
        B = other.value
        for i in range(len(A)):
            C_i = []
            row = A[i]
            for j in range(len(B[0])):
                col = []
                for ii in range(len(B)):
                    col.append(B[ii][j])
                C_i.append(self.dot(row,col))
            C.append(tuple(C_i))
        return Matrix(tuple(C))
    
    def __truediv__(self, other):
        return self * other.getInverse()
    
    def scalarMultiplication(self, other):
        A = self.value
        C = []
        for i in range(len(A)):
            C_i = []
            for j in range(len(A[0])):
                C_i.append(other*A[i][j])
            C.append(C_i)
        return Matrix(C)


    def __str__(self):
        A = self.value
        MatrixString = ""
        for i in range(len(A)):
            row = "|"
            for j in range(len(A[0])):
                row += " "+str(A[i][j])+" "
            MatrixString += row + "|\n"
        return MatrixString

    def dot(self, row, col):
        product = 0
        for i in range(len(row)):
            product += row[i]*col[i]
        return product
    
    def canMultiply(self, other):
        selfdimens = self.dimens.split("X")
        otherdimens = other.dimens.split("X")
        if(selfdimens[1]!=otherdimens[0]):
            return False
        else:
            return True

    def getDeterminant(self):
        A = list(self.value)
        determinant = 0
        if self.dimens=="2X2":
            determinant = self.get2x2Determinant(A)
            return determinant

        for focusColumn in list(range(len(A))):
            subMatrix = self.createSubMatrix(A, focusColumn)
            sign = (-1)**(focusColumn%2)
            subDeterminant = subMatrix.getDeterminant()
            determinant += sign * subDeterminant * A[0][focusColumn]
        return determinant

    def getInverse(self):
        if self.dimens == "2X2":
            return self.get2x2Inverse()
        detInv = self.getDeterminant()**-1
        C = self.getMatrixOfMinors().applyMatrixOfCofactors().getTranspose()* detInv
        return C     
    
    def get2x2Inverse(self):
        A = self.value
        detInv = self.getDeterminant()**-1
        return Matrix( ((A[1][1],-A[0][1]),(-A[1][0],A[0][0])) ) * detInv

    def getMatrixOfMinors(self):
        if self.dimens=="2X2":
            return self
        matrixMinors = []
        for i in range(len(self.value)):
            rowMinors = []
            for j in range(len(self.value[i])):
                matrixMinor = self.getMatrixMinor(i, j)
                rowMinors.append(matrixMinor.getDeterminant())
            matrixMinors.append(rowMinors)
        return Matrix(matrixMinors)

    def getMatrixMinor(self, i, j):
        m = self.value
        return Matrix(tuple([row[:j] + row[j+1:] for row in (m[:i]+m[i+1:])]))
    
    def applyMatrixOfCofactors(self):
        A = list(self.value)
        C = []
        for i in range(len(A)):
            Ci = []
            for j in range(len(A[i])):
                sign = ((-1)**i)*((-1)**j)
                Ci.append(sign*A[i][j])
            C.append(Ci)
        return Matrix(tuple(C))

    def get2x2Determinant(self, A):
        if self.dimens != "2X2":
            raise Exception("not 2x2 matrix")
        return A[0][0] * A[1][1] - A[1][0] * A[0][1]

    def createSubMatrix(self, A, focusColumn):
        AReduced = A[1:]
        for i in range(len(AReduced)): 
            AReduced[i] = AReduced[i][0:focusColumn] + AReduced[i][focusColumn+1:]
        return Matrix(AReduced)
    
    def getTranspose(self):
        A = self.value
        ATranspose = []
        for i in range(len(A[0])):
            ATRow = []
            for j in range(len(A)):
                ATRow.append(A[j][i])
            ATranspose.append(ATRow)

        return Matrix(ATranspose)
    
    def addMatrixToCsv(self):
        M = self.value
        with open('matrix.csv', mode='w') as matrix:
            writer = csv.writer(matrix, quoting=csv.QUOTE_ALL, lineterminator = '\n')
            for row in M:
                writer.writerow(row)
    
    def setMatrixFromCSV(self, csv_file_path):
        M = []
        with open(csv_file_path) as matrix:
            reader = csv.reader(matrix, delimiter=",")
            for row in reader:
                MRow = []
                for val in row:
                    MRow.append(int(val))
                M.append(MRow)
        self.value = tuple(M)
        return self

In [5]:
def TestGroupI2x2():
    tuple2x2a = ((1,1),(1,1))
    m2x2a = Matrix(tuple2x2a)
    print("size:"+str(m2x2a.size)+"\n")
    print("Arithmetic Tests")

    print(str(m2x2a)+"  PLUS \n"+str(m2x2a)+"  EQUALS ")
    m2x2b = m2x2a + m2x2a
    print(str(m2x2b))
    print("\n")
    
    print(str(m2x2b)+"  MINUS\n"+str(m2x2a)+"  EQUALS")
    print(str(m2x2b - m2x2a))
    print("\n\n")

    print(str(m2x2b)+"  TIMES\n"+str(m2x2a)+"  EQUALS")
    print(str(m2x2b * m2x2a))
    print("\n\n")

    vector = Matrix(((2,),(2,)))
    print(str(m2x2b)+"  TIMES\n"+str(vector)+"  EQUALS")
    print(str(m2x2b * vector))
    print("\n\n")

    tuple2x2C = ((4,7),(2,6))
    m2x2C = Matrix(tuple2x2C)
    print(str(m2x2C)+"  DIVIDED BY\n"+str(m2x2C)+"  EQUALS")
    print(str(m2x2C / m2x2C))


In [6]:
TestGroupI2x2()

size:(2, 2)

Arithmetic Tests
| 1  1 |
| 1  1 |
  PLUS 
| 1  1 |
| 1  1 |
  EQUALS 
| 2  2 |
| 2  2 |



| 2  2 |
| 2  2 |
  MINUS
| 1  1 |
| 1  1 |
  EQUALS
| 1  1 |
| 1  1 |




| 2  2 |
| 2  2 |
  TIMES
| 1  1 |
| 1  1 |
  EQUALS
| 4  4 |
| 4  4 |




| 2  2 |
| 2  2 |
  TIMES
| 2 |
| 2 |
  EQUALS
| 8 |
| 8 |




| 4  7 |
| 2  6 |
  DIVIDED BY
| 4  7 |
| 2  6 |
  EQUALS
| 1.0000000000000002  0.0 |
| 0.0  1.0000000000000002 |



In [7]:
def testGroupI3x3():

    tuple3x3a = ((1,1,1),(1,1,1),(1,1,1))
    print("Arithmetic Tests")
    m3x3a = Matrix(tuple3x3a)

    print(str(m3x3a)+"  PLUS \n"+str(m3x3a)+"  EQUALS ")
    m3x3b = m3x3a + m3x3a
    print(str(m3x3b))
    print("\n\n")
    
    print(str(m3x3b)+"  MINUS\n"+str(m3x3a)+"  EQUALS")
    print(str(m3x3b - m3x3a))
    print("\n\n")

    print(str(m3x3b)+"  TIMES\n"+str(m3x3a)+"  EQUALS")
    print(str(m3x3b * m3x3a))
    print("\n\n")

    vector = (Matrix(((2,),(2,),(2,))))
    print(str(m3x3b)+"  TIMES\n"+str(vector)+"  EQUALS")
    print(str(m3x3b * vector)) 
    print("\n\n")

    tuple3x3c = ((3,0,2),(2,0,-2),(0,1,1))
    m3x3c = Matrix(tuple3x3c)
    print(str(m3x3c)+"  DIVIDED BY\n"+str(m3x3c)+"  EQUALS")
    print(str(m3x3c/m3x3c))
    print("\n\n")   

In [8]:
testGroupI3x3()

Arithmetic Tests
| 1  1  1 |
| 1  1  1 |
| 1  1  1 |
  PLUS 
| 1  1  1 |
| 1  1  1 |
| 1  1  1 |
  EQUALS 
| 2  2  2 |
| 2  2  2 |
| 2  2  2 |




| 2  2  2 |
| 2  2  2 |
| 2  2  2 |
  MINUS
| 1  1  1 |
| 1  1  1 |
| 1  1  1 |
  EQUALS
| 1  1  1 |
| 1  1  1 |
| 1  1  1 |




| 2  2  2 |
| 2  2  2 |
| 2  2  2 |
  TIMES
| 1  1  1 |
| 1  1  1 |
| 1  1  1 |
  EQUALS
| 6  6  6 |
| 6  6  6 |
| 6  6  6 |




| 2  2  2 |
| 2  2  2 |
| 2  2  2 |
  TIMES
| 2 |
| 2 |
| 2 |
  EQUALS
| 12 |
| 12 |
| 12 |




| 3  0  2 |
| 2  0  -2 |
| 0  1  1 |
  DIVIDED BY
| 3  0  2 |
| 2  0  -2 |
| 0  1  1 |
  EQUALS
| 1.0  0.0  0.0 |
| 0.0  1.0  0.0 |
| 0.0  0.0  1.0 |






In [9]:
def testGroupII2x2():
    tuple2x2a = ((1,3),(-3,-1))
    m2x2a = Matrix(tuple2x2a)

    ("COMPLEX MATRIX OPERATIONS")
    print("DET\n"+str(m2x2a)+"  EQUALS\n")
    print(str(m2x2a.getDeterminant()))
    print("\n\n")


    tuple2x2b = ((4,7),(2,6))
    m2x2b = Matrix(tuple2x2b)

    print("ADJUGATE/ TRANSPOSE\n"+str(m2x2b)+"  EQUALS\n")
    m2x2c = m2x2b.getTranspose()
    print(m2x2c)
    
    detm2x2b = m2x2b.getDeterminant()
    print("DET OF\n"+str(m2x2b)+"  EQUALS")
    print(detm2x2b)
    print("\n")


    print("INVERSE OF\n"+str(m2x2b)+"  EQUALS\n")
    m2x2c = m2x2b.getInverse()
    print(m2x2c)
    print("see eg. at")
    print("https://www.mathsisfun.com/algebra/matrix-inverse-minors-cofactors-adjugate.html")
    print("\n")


In [10]:
testGroupII2x2()

DET
| 1  3 |
| -3  -1 |
  EQUALS

8



ADJUGATE/ TRANSPOSE
| 4  7 |
| 2  6 |
  EQUALS

| 4  2 |
| 7  6 |

DET OF
| 4  7 |
| 2  6 |
  EQUALS
10


INVERSE OF
| 4  7 |
| 2  6 |
  EQUALS

| 0.6000000000000001  -0.7000000000000001 |
| -0.2  0.4 |

see eg. at
https://www.mathsisfun.com/algebra/matrix-inverse-minors-cofactors-adjugate.html




In [11]:
def testGroupII3x3():
    tuple3x3a = ((1,3,2),(-3,-1,-3),(2,3,1))
    m3x3a = Matrix(tuple3x3a)

    ("COMPLEX MATRIX OPERATIONS")
    print("DET\n"+str(m3x3a)+"  EQUALS\n")
    print(str(m3x3a.getDeterminant()))
    print("\n\n")


    tuple3x3b = ((3,0,2),(2,0,-2),(0,1,1))
    m3x3b = Matrix(tuple3x3b)

    print("finding inverse of\n"+str(m3x3b))
    print("........................\n")

    print("M OF MINORS\n")
    m3x3c = m3x3b.getMatrixOfMinors()
    print(m3x3c)

    print("APPLY MATRIX OF COFACTORS\n")
    m3x3c = m3x3c.applyMatrixOfCofactors()
    print(str(m3x3c))

    print("ADJUGATE/ TRANSPOSE\n")
    m3x3c = m3x3c.getTranspose()
    print(m3x3c)
    
    detm3x3b = m3x3b.getDeterminant()
    print("DET OF\n"+str(m3x3b)+"  EQUALS")
    print(detm3x3b)
    print("\n")

    print(str(m3x3c)+"  DIVIDED BY\n"+  str(detm3x3b)+"\n  EQUALS\n")
    m3x3c = m3x3c*(detm3x3b)**-1
    print(m3x3c)
    print("\n")
    print("........................\n")

    print("INVERSE OF\n"+str(m3x3b)+"  EQUALS\n")
    m3x3c = m3x3b.getInverse()
    print(m3x3c)
    print("see eg. at")
    print("https://www.mathsisfun.com/algebra/matrix-inverse-minors-cofactors-adjugate.html")
    print("\n")


In [12]:
testGroupII3x3()

DET
| 1  3  2 |
| -3  -1  -3 |
| 2  3  1 |
  EQUALS

-15



finding inverse of
| 3  0  2 |
| 2  0  -2 |
| 0  1  1 |

........................

M OF MINORS

| 2  2  2 |
| -2  3  3 |
| 0  -10  0 |

APPLY MATRIX OF COFACTORS

| 2  -2  2 |
| 2  3  -3 |
| 0  10  0 |

ADJUGATE/ TRANSPOSE

| 2  2  0 |
| -2  3  10 |
| 2  -3  0 |

DET OF
| 3  0  2 |
| 2  0  -2 |
| 0  1  1 |
  EQUALS
10


| 2  2  0 |
| -2  3  10 |
| 2  -3  0 |
  DIVIDED BY
10
  EQUALS

| 0.2  0.2  0.0 |
| -0.2  0.30000000000000004  1.0 |
| 0.2  -0.30000000000000004  0.0 |



........................

INVERSE OF
| 3  0  2 |
| 2  0  -2 |
| 0  1  1 |
  EQUALS

| 0.2  0.2  0.0 |
| -0.2  0.30000000000000004  1.0 |
| 0.2  -0.30000000000000004  0.0 |

see eg. at
https://www.mathsisfun.com/algebra/matrix-inverse-minors-cofactors-adjugate.html




In [13]:
def testGroupIII():
    tuple3x3a = ((1,1,1),(1,1,1),(1,1,1))
    print("CSV TESTS")
    m3x3a = Matrix(tuple3x3a)

    print("Writing CSV")
    m3x3a.addMatrixToCsv()
    print('Reading CSV')
    print(Matrix().setMatrixFromCSV('matrix.csv'))

In [14]:
testGroupIII()

CSV TESTS
Writing CSV
Reading CSV
| 1  1  1 |
| 1  1  1 |
| 1  1  1 |



In [15]:
A = ((3,-1,-2),(-1,2,-2),(0,3,4))
B = ((-2,-4,4),(3,-3,4),(4,1,2))
a = Matrix(A)
b = Matrix(B)
print(a*b)

| -17  -11  4 |
| 0  -4  0 |
| 25  -5  20 |

